# Train Player Detection Model

In this notebook, we'll begin training the YOLOv8 model for player detection. The model will be trained on a custom dataset of soccer players, which includes images with bounding boxes around the players.

## Import necessary libraries

In [4]:
# Check GPU availability
!nvidia-smi

# Install dependencies
!pip3 install -q ultralytics roboflow

import os
from roboflow import Roboflow
from IPython.display import Image
import torch

# Set up directories
HOME = os.getcwd()
!mkdir -p {HOME}/datasets
!mkdir -p {HOME}/models

zsh:1: command not found: nvidia-smi
You should consider upgrading via the '/Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip' command.


## Download pre-trained model

We're going to be downloading Roboflow's pre-trained YOLOv8 soccer player detection model, which is trained on a custom dataset of soccer players. This model will be used as a starting point for our training.

We will be finetuning this model on hockey players, so it will be useful to have a model that is already trained on a similar task. 

In [5]:
# Download the soccer player detection model
!gdown -O "{HOME}/models/football-player-detection.pt" "https://drive.google.com/uc?id=17PXFNlx-jI7VjVo_vQnB1sONjRyvoB-q"

# Verify the model downloaded correctly
!ls -la {HOME}/models/

/Users/jetjadeja/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=17PXFNlx-jI7VjVo_vQnB1sONjRyvoB-q
From (redirected): https://drive.google.com/uc?id=17PXFNlx-jI7VjVo_vQnB1sONjRyvoB-q&confirm=t&uuid=a70bbbf6-c0fb-4171-b685-fa3cce5181c3
To: /Users/jetjadeja/Projects/work/sieve/hockey-vision-analytics/notebooks/models/football-player-detection.pt
100%|████████████████████████████████████████| 137M/137M [00:03<00:00, 43.9MB/s]
total 295936
drwxr-xr-x@ 3 jetjadeja  staff         96 Jun  3 11:09 .
drwxr-xr-x@ 5 jetjadeja  staff        160 Jun  3 11:03 ..
-rw-r--r--@ 1 jetjadeja  staff  136802409 Jul 25  2024 football-player-detection.pt


## Download datasets. 

In [9]:
import os
from dotenv import load_dotenv
from roboflow import Roboflow

# Load environment variables from .env file
load_dotenv()

# Get API key from environment variables
ROBOFLOW_API_KEY = os.getenv('ROBOFLOW_API_KEY')

if not ROBOFLOW_API_KEY:
    raise ValueError("ROBOFLOW_API_KEY not found in .env file. Please add your API key to the .env file.")

print("✅ Successfully loaded ROBOFLOW_API_KEY from .env file")

# Initialize Roboflow with API key
rf = Roboflow(api_key=ROBOFLOW_API_KEY)

# Download the hockey players dataset to the desired location
dataset_location = f"{HOME}/datasets/players"
project = rf.workspace("sportobal-xe8fu").project("hockey-players-i23se")
dataset = project.version(1).download("yolov8", location=dataset_location)

# Check the dataset structure
!cat {dataset_location}/data.yaml

✅ Successfully loaded ROBOFLOW_API_KEY from .env file
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to /Users/jetjadeja/Projects/work/sieve/hockey-vision-analytics/notebooks/datasets/players in yolov8:: 100%|██████████| 28884/28884 [00:03<00:00, 7489.49it/s]


names:
- Crowne
- Fitness
- Goalie
- Referee
nc: 4
roboflow:
  license: CC BY 4.0
  project: hockey-players-i23se
  url: https://universe.roboflow.com/sportobal-xe8fu/hockey-players-i23se/dataset/1
  version: 1
  workspace: sportobal-xe8fu
test: ../test/images
train: train/images
val: valid/images


### Inspect dataset

Let's now quickly inspect the dataset to ensure it has the correct format and contains the necessary annotations. The dataset should be in YOLO format, which consists of images and corresponding text files with bounding box annotations.

In [10]:
# Check how many images we have
!echo "Training images:"
!ls {dataset.location}/train/images | wc -l
!echo "Validation images:"
!ls {dataset.location}/valid/images | wc -l

# Look at the classes
import yaml
with open(f'{dataset.location}/data.yaml', 'r') as file:
    data_config = yaml.safe_load(file)
print("Classes:", data_config['names'])

Training images:
   12474
Validation images:
    1269
Classes: ['Crowne', 'Fitness', 'Goalie', 'Referee']


## Finetune the model

Now, let's finetune the pre-trained YOLOv8 model on our custom dataset of hockey players. We'll use the Roboflow library to handle the training process.

In [11]:
# Start training with transfer learning from soccer model
!yolo task=detect \
      mode=train \
      model={HOME}/models/football-player-detection.pt \
      data={dataset.location}/data.yaml \
      epochs=50 \
      imgsz=640 \
      batch=16 \
      patience=10 \
      freeze=10 \
      project={HOME}/training \
      name=hockey_players_v1 \
      exist_ok=True \
      amp=True \
      cache=True \
      lr0=0.001 \
      lrf=0.01

/Users/jetjadeja/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
Ultralytics 8.3.148 🚀 Python-3.9.6 torch-2.7.0 CPU (Apple M4 Pro)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=True, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/Users/jetjadeja/Projects/work/sieve/hockey-vision-analytics/notebooks/datasets/players/data.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=10, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=Fals

In [12]:
# While training is running, you can check the results
from IPython.display import Image, display

# View training curves (after a few epochs)
display(Image(f'{HOME}/mdoels/hockey_players_v1/results.png', width=800))

# Check confusion matrix
display(Image(f'{HOME}/hockey_training/hockey_players_v1/confusion_matrix.png', width=600))

FileNotFoundError: No such file or directory: '/Users/jetjadeja/Projects/work/sieve/hockey-vision-analytics/notebooks/hockey_training/hockey_players_v1/results.png'

FileNotFoundError: No such file or directory: '/Users/jetjadeja/Projects/work/sieve/hockey-vision-analytics/notebooks/hockey_training/hockey_players_v1/results.png'

<IPython.core.display.Image object>

FileNotFoundError: No such file or directory: '/Users/jetjadeja/Projects/work/sieve/hockey-vision-analytics/notebooks/hockey_training/hockey_players_v1/confusion_matrix.png'

FileNotFoundError: No such file or directory: '/Users/jetjadeja/Projects/work/sieve/hockey-vision-analytics/notebooks/hockey_training/hockey_players_v1/confusion_matrix.png'

<IPython.core.display.Image object>